# Exercise 6: Propagating parameter uncertainty

We saw in Exercise 5 that there can be more than 1 parameter set that provides a good match the the data. Rather than only selecting the best-fitting set, it's possible to select a few different parameter sets that provide relatively close match to the data and propagate these forward in time. This is one possible way of captuing parameter uncertainty.

In [ ]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import sciris as sc

# Read in the data
flu = pd.read_csv("flu_cases.csv")

# How many days of observation do we have when we need to make a decision?
n_obs = 5

# What do the data look like at this point?
pl.scatter(flu['day'][:n_obs], flu['cases'][:n_obs], label='Data')
pl.title('Number of infected students')
pl.xlabel('Day')
pl.xticks(np.arange(n_obs))
pl.show()

# Define the parameter ranges for our model
betas = np.linspace(0.1, 0.9, 9)
gammas = np.array([1/48, 1/24, 1/12, 1/6, 1/3, 1/1.5])

# Other inputs
npts = 100  # How many data points in total
I0 = 1  # Number of initially infected people
N = 1000  # Total population size (assuming a closed population)
dt = 1  # Timestep

# Make arrays where we will store the estimates
x = np.arange(npts)
S = np.zeros(npts)
I = np.zeros(npts)
R = np.zeros(npts)

# We will store the number of infected people and mismatch with the data for each parameter combination
I_full = np.empty((npts, len(betas)*len(gammas)))
mismatch = np.empty(len(betas)*len(gammas))

# Initial conditions
S[0] = N - I0
I[0] = I0

# Simulate the model over time
idx = 0
for beta in betas:
    for gamma in gammas:

        for t in x[:-1]:
            infections = beta * S[t] * I[t]/N * dt
            recoveries = gamma * I[t] * dt
            S[t + 1] = S[t] - infections
            I[t + 1] = I[t] + infections - recoveries
            R[t + 1] = R[t] + recoveries

        I_full[:, idx] = I
        mismatch[idx] = sum((flu['cases'][:n_obs]-I[:n_obs])**2)

        idx += 1

# Make some things needed for plotting
time = x * dt
colors = sc.vectocolor(mismatch)  # Use to color the lines according to mismatch

# Plot the model estimates of the number of infections alongside the data
pl.scatter(time[:n_obs], flu['cases'][:n_obs], label='Data')
for i in range(len(betas)*len(gammas)):
    pl.plot(time[:n_obs], I_full[:n_obs, i], c=colors[i], alpha=0.5, lw=1)
pl.title('Number of infected students')
pl.xlabel('Day')
pl.xticks(np.arange(n_obs))
pl.show()

# Project the models with the lowest mismatch forward in time
n_for = 10  # number of points to forecast
mmt = 10  # short for mismatch threshold - we will forecast anything where the mismatch is lower than this
pl.scatter(time[:n_for], flu['cases'][:n_for], label='Data')
for i in range(len(betas)*len(gammas)):
    if mismatch[i] < mmt:
        pl.plot(time[:n_for], I_full[:n_for, i], c='g', lw=1)
    else:
        pl.plot(time[:n_obs], I_full[:n_obs, i], c='r', alpha=0.5, lw=1)

pl.title('Number of infected students')
pl.xlabel('Day')
pl.xticks(np.arange(n_for))
pl.show()

We see that our central projection is still too high, but the data points lie within the bounds of what was considered consistent with the data. Note that there is a large range, with 10-120 cases by day 9.
